# Import modules

In [1]:
import os
import zipfile
import torch
import torchvision
from torchvision import datasets, transforms
import random
from tqdm.auto import tqdm
from pathlib import Path
import shutil
from typing import Tuple, Dict
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')

2024-05-09 22:23:04.234797: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 22:23:04.234930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 22:23:04.413628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Download dataset

In [2]:
data_dir = Path('data/')

if data_dir.is_dir():
    print('Folder already exists')
else:
    # Downloading subset of food101
    print('Downloading subset of Food101 dataset')
    data_dir.mkdir(parents=True,exist_ok=True)
    !kaggle datasets download -d satish1v/food101subset -p data/
    zip_path = data_dir / 'food101subset.zip'
    with zipfile.ZipFile(zip_path) as zipref:
        zipref.extractall(data_dir)
    os.remove(zip_path)

    # Keeping only top 3 food subfolders instead of 101
    sub_folders = os.listdir(data_dir)
    for sub_folder in [sub_folder for sub_folder in sub_folders[3:] if (data_dir/sub_folder).is_dir()]:
        sub_folder_path = data_dir / sub_folder
        shutil.rmtree(sub_folder_path)

Dataset URL: https://www.kaggle.com/datasets/satish1v/food101subset
License(s): CC0-1.0
 98%|████████████████████████████████████████▎| 939M/956M [00:08<00:00, 187MB/s]
100%|█████████████████████████████████████████| 956M/956M [00:08<00:00, 123MB/s]


# Experimentation setup

In [3]:
model_names = ['resnet18','alexnet','vgg11']

batch_sizes = [100,50]

dropouts = [0.3,0.5]

# Helper functions

In [4]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device:torch.device) -> Tuple[float,float,float,float,float]:
    # train mode
    model.train()

    # Set up metrics
    train_loss, train_acc, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0

    # Loop through batches
    for X, y in dataloader:

        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        logits = model(X)

        # 2. Compute loss
        loss = loss_fn(logits, y)

        # 3. Zero gradients
        optimizer.zero_grad()

        # 4. Backward propagation
        loss.backward()

        # 5. Update weights and biases
        optimizer.step()

        # Get the most confident predictions
        predictions = torch.argmax(logits,dim=1).cpu()
        y = y.cpu()

        # Calculate metrics
        train_loss += loss.item()
        train_acc += accuracy_score(predictions, y)
        train_precision += precision_score(predictions, y, average='macro')
        train_recall += recall_score(predictions, y, average='macro')
        train_f1 += f1_score(predictions, y, average='macro')

    # Averaging metrics
    train_loss = round(train_loss/len(dataloader),2)
    train_acc = round(train_acc/len(dataloader),2)
    train_precision = round(train_precision/len(dataloader),2)
    train_recall = round(train_recall/len(dataloader),2)
    train_f1 = round(train_f1/len(dataloader),2)

    return train_loss, train_acc, train_precision, train_recall, train_f1


def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              device:torch.device) -> Tuple[float,float,float,float,float]:

    # Set up metrics
    test_loss, test_acc, test_precision, test_recall, test_f1 = 0, 0, 0, 0, 0

    # Turn off updating gradients
    model.eval()
    with torch.inference_mode():
        # Loop through batches
        for X, y in dataloader:

            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            logits = model(X)

            # 2. Compute loss
            loss = loss_fn(logits, y)

            # Get the most confident predictions
            predictions = torch.argmax(logits,dim=1).cpu()
            y = y.cpu()

            # Calculate metrics
            test_loss += loss.item()
            test_acc += accuracy_score(predictions, y)
            test_precision += precision_score(predictions, y, average='macro')
            test_recall += recall_score(predictions, y, average='macro')
            test_f1 += f1_score(predictions, y, average='macro')

    # Averaging metrics
    test_loss = round(test_loss/len(dataloader),2)
    test_acc = round(test_acc/len(dataloader),2)
    test_precision = round(test_precision/len(dataloader),2)
    test_recall = round(test_recall/len(dataloader),2)
    test_f1 = round(test_f1/len(dataloader),2)

    return test_loss, test_acc, test_precision, test_recall, test_f1


def create_writer(model_name:str, batch_size:int, dropout_rate:float) -> SummaryWriter:
    log_dir = Path("experiments") / model_name / f'BS{batch_size}' / f'DR{int(dropout_rate*100)}perc'
    return SummaryWriter(log_dir=log_dir)


def remove_empty_logs(model_name:str, batch_size:int, dropout_rate:float):
    log_dir = Path("experiments") / model_name / f'BS{batch_size}' / f'DR{int(dropout_rate*100)}perc'
    empty_logs = [file for file in os.listdir(log_dir) if (log_dir/file).is_file()]
    for empty_log in empty_logs:
        os.remove(log_dir/empty_log)


def train(model:torch.nn.Module,
            train_dataloader:torch.utils.data.DataLoader,
            test_dataloader:torch.utils.data.DataLoader,
            loss_fn:torch.nn.Module,
            optimizer: torch.optim.Optimizer,
            device:torch.device,
            model_name:str,
            batch_size:int,
            dropout_rate:float,
            epochs:int,
            ):

    lowest_test_loss = float('inf')
    for epoch in tqdm(range(1,epochs+1)):
        # Train step
        train_loss, train_acc, train_precision, train_recall, train_f1 = train_step(model,train_dataloader,loss_fn,optimizer,device)
        print(f'EPOCH_{epoch}:\n \
            Train metrics: loss = {train_loss}; acc = {train_acc}; precision = {train_precision}; recall = {train_recall}; f1 = {train_f1}')

        # Test step
        test_loss, test_acc, test_precision, test_recall, test_f1 = test_step(model,test_dataloader,loss_fn,device)
        print(f'EPOCH_{epoch}:\n \
            Test metrics: loss = {test_loss}; acc = {test_acc}; precision = {test_precision}; recall = {test_recall}; f1 = {test_f1}')

        writer = create_writer(model_name, batch_size, dropout_rate)

        # Add results to SummaryWriter
        writer.add_scalars(main_tag='CrossEntropyLoss',
                           tag_scalar_dict={'train':train_loss,
                                             'test':test_loss},
                            global_step=epoch)
        writer.add_scalars(main_tag='Accuracy',
                           tag_scalar_dict={'train':train_acc,
                                             'test':test_acc},
                            global_step=epoch)
        writer.add_scalars(main_tag='Precision',
                           tag_scalar_dict={'train':train_precision,
                                             'test':test_precision},
                            global_step=epoch)
        writer.add_scalars(main_tag='Recall',
                           tag_scalar_dict={'train':train_recall,
                                             'test':test_recall},
                            global_step=epoch)

        writer.add_scalars(main_tag='F1',
                            tag_scalar_dict={'train':train_f1,
                                                'test':test_f1},
                            global_step=epoch)

        writer.close()
        
        # Removing unnecessary files in log_dir
        remove_empty_logs(model_name, batch_size, dropout_rate)

        # Checking if test_loss decreased
        if test_loss<lowest_test_loss:
            model_path = Path('models/') / f'{model_name}_BS{batch_size}_DR{int(dropout_rate*100)}perc.pt'
            Path('models/').mkdir(parents=True,exist_ok=True)
            print(f'\n{"-"*80}\nTest loss decreased, saving model to "{model_path}"\n{"-"*80}\n')
            torch.save(model, model_path)
            lowest_test_loss = test_loss

# Settings

In [5]:
# Set the seed for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

train_threshold = 0.8
out_features = len(os.listdir(data_dir))
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.05
epochs = 10

experiment_dir = Path('experiments/')
experiment_dir.mkdir(parents=True,exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Experimenting by iterating through "experimentation setup"

In [6]:
for model_name in model_names:

    if model_name=='alexnet':
        weights = torchvision.models.AlexNet_Weights.DEFAULT
        model = torchvision.models.alexnet(weights=weights)

    elif model_name=='vgg11':
        weights = torchvision.models.VGG11_Weights.DEFAULT
        model = torchvision.models.vgg11(weights=weights)

    elif model_name=='resnet18':
        weights = torchvision.models.ResNet18_Weights.DEFAULT
        model = torchvision.models.resnet18(weights=weights)
        # Making the last layer similar to first two models for purposes of experiment
        model.classifier = model.fc
        del model.fc
        model.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=random.random()),
            model.classifier
        )
        
        # Overwriting _forward_impl method for resnet18 to change "fc" to "classifier"
        def custom_resnet18_forward_impl(self, x:torch.Tensor) -> torch.Tensor:
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.maxpool(x)

            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)

            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)

            return x
        
        # Patch the _forward_impl method
        model._forward_impl = custom_resnet18_forward_impl.__get__(model, torchvision.models.ResNet)


    model.classifier[-1] = torch.nn.Linear(in_features=model.classifier[-1].in_features,
                                            out_features=out_features, bias=True)

    # Get model's default transform
    auto_transforms = weights.transforms()

    # Create datasets
    dataset = datasets.ImageFolder(root=data_dir, transform=auto_transforms)
    train_size = int(train_threshold * len(dataset))
    test_size = int(len(dataset) - train_size)
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    for batch_size in batch_sizes:
        # Create dataloaders
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        for dropout in dropouts:
            # Changing dropout rate
            for i, layer in enumerate(model.classifier):
                if type(layer)==torch.nn.modules.dropout.Dropout:
                    model.classifier[i] = torch.nn.Dropout(p=dropout, inplace=False)

            # Putting only classifier params equal to freezing feature extraction layers
            optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)

            # Train, save the best model, track metrics
            model.to(device)
            train(model,train_dataloader,test_dataloader,
                loss_fn,optimizer,device,model_name,
                batch_size,dropout,epochs)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 113MB/s]


  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH_1:
             Train metrics: loss = 10.32; acc = 0.34; precision = 0.38; recall = 0.23; f1 = 0.25
EPOCH_1:
             Test metrics: loss = 8.94; acc = 0.3; precision = 0.33; recall = 0.1; f1 = 0.15

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/resnet18_BS100_DR30perc.pt"
--------------------------------------------------------------------------------

EPOCH_2:
             Train metrics: loss = 5.29; acc = 0.52; precision = 0.52; recall = 0.54; f1 = 0.43
EPOCH_2:
             Test metrics: loss = 3.03; acc = 0.56; precision = 0.57; recall = 0.65; f1 = 0.49

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/resnet18_BS100_DR30perc.pt"
--------------------------------------------------------------------------------

EPOCH_3:
             Train metrics: loss = 3.92; acc = 0.6; precision = 0.6; recall = 0.65; f1 = 0.52
EPOCH_

  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH_1:
             Train metrics: loss = 5.42; acc = 0.56; precision = 0.6; recall = 0.71; f1 = 0.52
EPOCH_1:
             Test metrics: loss = 3.94; acc = 0.53; precision = 0.51; recall = 0.75; f1 = 0.48

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/resnet18_BS100_DR50perc.pt"
--------------------------------------------------------------------------------

EPOCH_2:
             Train metrics: loss = 1.9; acc = 0.79; precision = 0.77; recall = 0.84; f1 = 0.77
EPOCH_2:
             Test metrics: loss = 2.32; acc = 0.67; precision = 0.69; recall = 0.77; f1 = 0.62

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/resnet18_BS100_DR50perc.pt"
--------------------------------------------------------------------------------

EPOCH_3:
             Train metrics: loss = 1.58; acc = 0.81; precision = 0.8; recall = 0.84; f1 = 0.8
EPOCH_3

...

EPOCH_1:
             Train metrics: loss = 507.54; acc = 0.95; precision = 0.96; recall = 0.96; f1 = 0.95
EPOCH_1:
             Test metrics: loss = 1995.6; acc = 0.91; precision = 0.9; recall = 0.92; f1 = 0.9

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/vgg11_BS100_DR50perc.pt"
--------------------------------------------------------------------------------

EPOCH_2:
             Train metrics: loss = 295.46; acc = 0.97; precision = 0.97; recall = 0.97; f1 = 0.97
EPOCH_2:
             Test metrics: loss = 1907.95; acc = 0.92; precision = 0.92; recall = 0.92; f1 = 0.91

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/vgg11_BS100_DR50perc.pt"
--------------------------------------------------------------------------------

EPOCH_3:
             Train metrics: loss = 540.61; acc = 0.97; precision = 0.97; recall = 0.97; f1 = 0.97


  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH_1:
             Train metrics: loss = 1555.16; acc = 0.97; precision = 0.97; recall = 0.97; f1 = 0.97
EPOCH_1:
             Test metrics: loss = 8277.13; acc = 0.85; precision = 0.87; recall = 0.88; f1 = 0.86

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/vgg11_BS50_DR30perc.pt"
--------------------------------------------------------------------------------

EPOCH_2:
             Train metrics: loss = 1988.1; acc = 0.96; precision = 0.96; recall = 0.96; f1 = 0.96
EPOCH_2:
             Test metrics: loss = 3911.37; acc = 0.94; precision = 0.94; recall = 0.94; f1 = 0.94

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/vgg11_BS50_DR30perc.pt"
--------------------------------------------------------------------------------

EPOCH_3:
             Train metrics: loss = 1119.07; acc = 0.98; precision = 0.98; recall = 0.98; f1 = 0.

  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH_1:
             Train metrics: loss = 329.52; acc = 0.99; precision = 0.99; recall = 0.99; f1 = 0.99
EPOCH_1:
             Test metrics: loss = 13394.49; acc = 0.92; precision = 0.92; recall = 0.92; f1 = 0.92

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/vgg11_BS50_DR50perc.pt"
--------------------------------------------------------------------------------

EPOCH_2:
             Train metrics: loss = 41.56; acc = 1.0; precision = 1.0; recall = 1.0; f1 = 1.0
EPOCH_2:
             Test metrics: loss = 15024.05; acc = 0.93; precision = 0.93; recall = 0.93; f1 = 0.93
EPOCH_3:
             Train metrics: loss = 2005.44; acc = 0.99; precision = 0.99; recall = 0.99; f1 = 0.99
EPOCH_3:
             Test metrics: loss = 8791.05; acc = 0.94; precision = 0.94; recall = 0.94; f1 = 0.94

--------------------------------------------------------------------------------
Test loss decreased, saving model to "models/

# Checking metrics in tensorboard

In [8]:
%load_ext tensorboard
%tensorboard --logdir experiments